# Siamese Network
Neural network is almost good at every problem but they tend to rely on the need of more data or batter quality data. Problem like face recognition and signature verification, we can't always rely on more data. To solve this problem a new type of neural network is used called Siamese Networks. It only use few numbers of images to get better predictions, it has the ability to learn from very little data.

## What are Siamese Network
Is a class of neural network architecture that contain two or more identical subnetworks. "identical" mean they have same configuration with the same parameters and weight. Parameter updating is mirrored across sub-networks. It is used to find the similarityof the inputs by comapring its features vectors. <br><br>
Traditionally normal NN learns to predict multiple classes, this can be problematic when adding or removing new classes to the adta, in this case we have to update the NN and retrain the whole dataset. Also Deep neural network need large volume of data to train on. SNN (Siamese Neural Network) on the other hand only learn similarity function. Thus we can train it to see of two image are the same. This enables us to classify new classes of data without training the network again.

## Pros and Cons of SNN
**Pros:**
1. More robust to class imbalance, with the aid of one-shot learning, given a few images per class is sufficient for SNN to recognize images in the futures.
2. Nice to an ensemble with the best classifier, given the learning mechanism it is somewhat different from classification, simple averaging of it with a classifier can do much better than average 2 correlated supervised model (e.g. GBM & RF classifier).
3. Learning from semantic similarity, siamese focuses on learning embedding (in the deeper layer) that place the same classes/concepts close together. Hence can learn semantic similarity.
<br><br>

**Cons:**
1. Needs more training time than normal networks, since siamese networks involves quadratic pairs to learn from (to see all information available) it is slower than normal classification type of learning(pointwise learning)
2. Doesn't output probabilities, since training involves pairwise learning, it won't output the probabilities of the predicition but the distance from each class.

## Image Classification Model using Traditional Deep Learning Neural Network Architecture
You first need to obtained labeled dataset containing images of dogs and cats. After training the neural network, and upon gicing any input image the network can only output labels as dog or cat. This is standard computer vision problem known as Image Classification.<br>
In classification, the image is given into the NN and the output layer give the list of probability distribution over all classes (using softmax or other activation function as per the classification problem being solved).<br>
For example i want to classify if this image is cat ot not then for every inpu image we generate two probabilities indicating the probability of the image belonging to each of the 2 classes, but when training we require a lot of images of each class and the model can only predict from the trained labels. <br>
<span style="background-color: #FFFFED">One problem where we might come across is when we want to create employee attendance systems, this the cost of data collection and re-training is high each time a new class is added or a new employee joins</span> <br>

Source:<br>
- [A Comprehensive Guide to Siamese Neural Networks by Rinki Nag](https://medium.com/@rinkinag24/a-comprehensive-guide-to-siamese-neural-networks-3358658c0513)
- [A friendly introduction to Siamese Networks by Sean Benhur](https://towardsdatascience.com/a-friendly-introduction-to-siamese-networks-85ab17522942)

tanks 4 readin dis <3